In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('./utils')
sys.path.append('./utils/APIs')
from utils.common import save_model, write_to_file
import matplotlib.pyplot as plt
import torch
import timm
import argparse
from Config import config
from Trainer import Trainer
from Models.OTEModel import Model,MAMLModel

from torch.utils.data import DataLoader
from dataManagement.DatasetHelper import DatasetHelper
from dataManagement.DatasetLoader import DatasetLoader
from dataManagement.CustomDataset import CustomDataset

2025-02-24 13:06:18.041385: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-24 13:06:18.059238: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740373578.080848    2257 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740373578.087628    2257 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-24 13:06:18.110746: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# 训练模型
do_train = True
# 预测测试集数据
do_test = True
config.num_words_to_keep = 3000
config.batch_size = 32
config.num_words_x_doc = 100
config.lr = 0.001
config.momentum = 0.9
config.wd = 1e-4
config.load_model_path = './save_models/CMAT/pytorch_model.bin'
config.fuse_model_type = 'CMAT'
config.epoch = 20
config.pre_train_epoch = 30

#仅用文本预测
text_only = False
#仅用图像预测
img_only = False
config.only = 'img' if img_only else None
config.only = 'text' if text_only else None
if img_only and text_only: config.only = None

In [3]:
# Initilaztion
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
backbone = Model(num_classes=config.num_labels, vocab_size=3000, embedding_size=512)
model = MAMLModel(backbone, config.num_labels)
trainer = Trainer(config, model, device)

In [5]:
# PreTrain
def preTrain():
    task_lr = 0.01
    # inner_steps = 5
    inner_steps = 3
    for e in range(config.pre_train_epoch):
        print('-' * 20 + ' ' + 'PreTrain Epoch ' + str(e+1) + ' ' + '-' * 20)
        trainer.preTrain(train_loader, e, task_lr, inner_steps)
        print()

In [6]:
# Train
def train():
    best_acc = 0
    epoch = config.epoch
    for e in range(epoch):
        print('-' * 20 + ' ' + 'Epoch ' + str(e+1) + ' ' + '-' * 20)
        trainer.train(train_loader,e)
        test_acc = trainer.valid(val_loader,e)
        if test_acc > best_acc:
            best_acc = test_acc
            save_path = './save_models'
            save_model(save_path, config.fuse_model_type, model)
            print('Update best model!')
        print()

In [7]:
# Test
def test():
    test_file_path = '../dataset/CornDataset/csv/test/test_data.csv'
    
    data_loader = DatasetLoader()
    data_loader.load_data(test_file_path,True)
    test_data = data_loader.get_test_data()

    data_helper = DatasetHelper(config.num_words_to_keep)
    test_y = data_helper.preprocess_labels(test_data, None)
    test_i = data_helper.preprocess_images(test_data.get_images(), None)
    test_t = data_helper.preprocess_texts(test_data.get_texts(), None, config.num_words_x_doc)

    data_loader.set_test_data(test_y, test_i, test_t)

    # get CustomDataset
    test_custom_dataset = CustomDataset(data_loader.get_test_data())
    test_loader = DataLoader(test_custom_dataset, config.batch_size, shuffle=True)
    
    if config.load_model_path is not None:
        print("model load successfully")
        model.load_state_dict(torch.load(config.load_model_path))

    trainer.predict(test_loader)

In [ ]:
# Dataset And K-CV
import pandas as pd
import numpy as np
dataset_file_path = '../dataset/CornDataset/csv/train/train_data.csv'
def read_from_file(path,only=None):
    data = []
    labels = []
    df = pd.read_csv(path)
    for index, row in df.iterrows():
        image_path, describe, label = row['image'], row['describe'], row['label']
        data.append((image_path, describe))
        labels.append(label)
    return data,labels
data,label = read_from_file(dataset_file_path)
from sklearn.model_selection import  StratifiedKFold
skf = StratifiedKFold(n_splits=5) #5折
for i, (train_idx, val_idx) in enumerate(skf.split(data,label)):
    train_data, val_data = np.array(data)[train_idx],np.array(data)[val_idx]
    train_label, val_label = np.array(label)[train_idx],np.array(label)[val_idx]

    data_loader = DatasetLoader()
    data_loader.train.load_data((train_data,train_label),'Train')
    data_loader.val.load_data((val_data,val_label),'Val')

    data_loader.set_train_data(data_loader.train.get_labels(), data_loader.train.get_images(), data_loader.train.get_texts())
    data_loader.set_val_data(data_loader.val.get_labels(), data_loader.val.get_images(), data_loader.val.get_texts())
    
    train_data = data_loader.get_train_data()
    val_data = data_loader.get_val_data()
    
    data_helper = DatasetHelper(config.num_words_to_keep)
    train_y, val_y = data_helper.preprocess_labels(train_data, val_data)
    train_i, val_i = data_helper.preprocess_images(train_data.get_images(), val_data.get_images())
    train_t, val_t = data_helper.preprocess_texts(train_data.get_texts(), val_data.get_texts(), config.num_words_x_doc)
    
    # labels、images、text  set to data_geter
    data_loader.set_train_data(train_y, train_i, train_t)
    data_loader.set_val_data(val_y, val_i, val_t)
    
    # get CustomDataset (train and val)
    train_custom_dataset = CustomDataset(data_loader.get_train_data())
    val_custom_dataset = CustomDataset(data_loader.get_val_data())
    
    train_loader = DataLoader(train_custom_dataset, config.batch_size, shuffle=True)
    val_loader = DataLoader(val_custom_dataset, config.batch_size, shuffle=False)
    
    backbone = Model(num_classes=config.num_labels, vocab_size=3000, embedding_size=512)
    model = MAMLModel(backbone, config.num_labels)
    trainer = Trainer(config, model, device)
    # preTrain()
    train()
    test()